In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
from tqdm import tqdm
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Download stat and ops files

In [12]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/',
    search_pattern_re='stat.*npy',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230531/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230531/scanimage_data/zstack/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230420/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230523/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230523/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230523/scanimage_data/zstack/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322R/20230517/scanimage_data/baseline/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/m

In [13]:
paths_found = [p for p in paths_found if 'exp' in p]

In [14]:
dates_toUse = natsort.natsorted([Path(p).parts[-7] for p in paths_found])
dates_toUse

['20230419',
 '20230420',
 '20230424',
 '20230425',
 '20230426',
 '20230427',
 '20230428',
 '20230429',
 '20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230530',
 '20230531']

In [18]:
[sftp.get(str(Path(p).parent / 'stat.npy'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/stat_and_ops') / Path(p).parts[-7] / 'stat.npy')) for p in paths_found[2:]];

100%|████████████████████████████████████████████████████████████████████████▉| 19.9M/19.9M [00:02<00:00, 8.20Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 18.9M/18.9M [00:00<00:00, 20.5Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 17.8M/17.8M [00:00<00:00, 20.7Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 17.5M/17.5M [00:00<00:00, 28.5Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 19.9M/19.9M [00:00<00:00, 32.2Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 20.8M/20.9M [00:00<00:00, 35.9Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 19.8M/19.8M [00:00<00:00, 37.3Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 20.8M/20.8M [00:00<00:00, 40.2Mb/s]
100%|███████████████████████████████████████████████████████████████████

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [15]:
[sftp.get(str(Path(p).parent / 'ops.npy'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/stat_and_ops') / Path(p).parts[-7] / 'ops.npy')) for p in paths_found[2:]];

100%|██████████████████████████████████████████████████████████████████████████▉| 231M/231M [00:07<00:00, 29.2Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 232M/232M [00:03<00:00, 60.5Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 232M/232M [00:04<00:00, 56.6Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 231M/231M [00:04<00:00, 56.3Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 232M/232M [00:04<00:00, 56.9Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 231M/231M [00:04<00:00, 57.1Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 233M/233M [00:04<00:00, 54.8Mb/s]
100%|██████████████████████████████████████████████████████████████████████████▉| 232M/232M [00:05<00:00, 46.2Mb/s]
100%|███████████████████████████████████████████████████████████████████

# Download logger files

In [19]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/',
    search_pattern_re='logger\.mat',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230531/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230420/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230523/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230517/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230515/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230430/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230512/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230511/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230514/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/20230504/analysis_data/logger.mat
/n/files/Neurobio/MI

In [20]:
dates_toUse = natsort.natsorted([Path(p).parts[-3] for p in paths_found])
dates_toUse

['20230419',
 '20230420',
 '20230424',
 '20230425',
 '20230426',
 '20230427',
 '20230428',
 '20230429',
 '20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230530',
 '20230531']

In [21]:
[sftp.get(str(Path(p).parent / 'logger.mat'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/loggers') / date / 'logger.mat')) for date, p in zip(dates_toUse, paths_found)];

100%|████████████████████████████████████████████████████████████████████████▋| 4.82M/4.84M [00:01<00:00, 4.40Mb/s]
100%|████████████████████████████████████████████████████████████████████████▊| 4.82M/4.83M [00:00<00:00, 8.48Mb/s]
100%|████████████████████████████████████████████████████████████████████████▋| 4.78M/4.80M [00:00<00:00, 11.9Mb/s]
 99%|████████████████████████████████████████████████████████████████████████▌| 4.82M/4.85M [00:00<00:00, 11.6Mb/s]
100%|████████████████████████████████████████████████████████████████████████▊| 13.4M/13.5M [00:00<00:00, 16.0Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 13.4M/13.4M [00:00<00:00, 21.9Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 13.5M/13.5M [00:00<00:00, 23.3Mb/s]
100%|████████████████████████████████████████████████████████████████████████▊| 13.5M/13.5M [00:00<00:00, 26.5Mb/s]
100%|███████████████████████████████████████████████████████████████████

In [22]:
[sftp.get(str(Path(p).parent / 'expParams.mat'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/loggers') / date / 'expParams.mat')) for date, p in zip(dates_toUse, paths_found)];

100%|████████████████████████████████████████████████████████████████████████▉| 39.1M/39.1M [00:00<00:00, 53.0Mb/s]
100%|████████████████████████████████████████████████████████████████████████▉| 39.0M/39.0M [00:00<00:00, 47.4Mb/s]
 95%|██████████████████████████████████████████████████████████████████████▋   | 65.5k/68.6k [00:00<00:00, 980kb/s]
 95%|██████████████████████████████████████████████████████████████████████▍   | 65.5k/68.8k [00:00<00:00, 565kb/s]
 99%|████████████████████████████████████████████████████████████████████████▌| 4.13M/4.15M [00:00<00:00, 21.0Mb/s]
 99%|████████████████████████████████████████████████████████████████████████▍| 4.29M/4.32M [00:00<00:00, 20.8Mb/s]
 99%|████████████████████████████████████████████████████████████████████████▌| 4.10M/4.12M [00:00<00:00, 15.9Mb/s]
 99%|████████████████████████████████████████████████████████████████████████▌| 4.10M/4.12M [00:00<00:00, 21.5Mb/s]
100%|███████████████████████████████████████████████████████████████████

# Download wavesurfer files

In [ ]:
paths_files_remote = sftp_t.search_recursive(
    path=r'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/',
    search_pattern_re=r'.*exp.*h5',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_files_remote = natsort.natsorted(paths_files_remote)

In [ ]:
dates_files_remote = [Path(p).parent.parent.name for p in paths_files_remote]

In [ ]:
dates_files_remote

In [ ]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/wavesurfer_files/'

In [ ]:
[sftp_t.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_files_remote, dates_files_remote)]

# Download camera CSV files

In [ ]:
paths_csvs_remote = sftp_t.search_recursive(
    path=r'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0322/mouse_0322R/',
    search_pattern_re=r'.*cam4.*csv',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_csvs_remote = natsort.natsorted(paths_csvs_remote)

In [ ]:
dates_csvs_remote = [Path(p).parent.parent.parent.parent.name for p in paths_csvs_remote]

In [ ]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/cam4_CSVs/'

In [ ]:
[sftp_t.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_csvs_remote, dates_csvs_remote)]

# Download VQT files

In [ ]:
paths_csvs_remote = sftp_t.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/',
    search_pattern_re=r'.*cam4.*csv',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_csvs_remote = natsort.natsorted(paths_csvs_remote)

In [ ]:
dates_csvs_remote = [Path(p).parent.parent.parent.parent.name for p in paths_csvs_remote]

In [ ]:
directory_local_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322R/cam4_CSVs/'

In [ ]:
[sftp_t.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_csvs_remote, dates_csvs_remote)]

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()